## Data Analysis

### Data We Have

We have the following data:

 * `data/statements/contest_[en/ro].json` - the statements of the problems in the contest.
 * `data/solutions/solutions_[en/ro]_round_[1/2/3].json` - the solutions of the problems in the contest. We have 3 rounds for accounting for variations in the LLMs' answers.

In [1]:
import matplotlib.pyplot as plt
import duckdb
import glob

import internal_types

In [22]:
# For simplicity, only look at a single langauge for now.
LANG = "en"

# Load the data
statements_nested = duckdb.read_json(internal_types.get_statement_files()[LANG])
statements = duckdb.sql(
    "SELECT name as edition, generate_subscripts(problems, 1) - 1 AS problem_index, unnest(problems, recursive := true) FROM statements_nested"
)
solutions = duckdb.read_json(internal_types.get_solutions_files_glob()[LANG])
data = duckdb.sql(
    "SELECT * FROM statements NATURAL JOIN solutions"
)

In [33]:
duckdb.sql("SELECT COUNT(python_code) from data")

┌────────────────────┐
│ count(python_code) │
│       int64        │
├────────────────────┤
│                538 │
└────────────────────┘

In [35]:
duckdb.sql("SELECT llm, COUNT(*) correct_answers, COUNT(python_code) FROM data WHERE answer == correct_answer GROUP BY llm")

┌───────────────────────────────────┬─────────────────┬────────────────────┐
│                llm                │ correct_answers │ count(python_code) │
│              varchar              │      int64      │       int64        │
├───────────────────────────────────┼─────────────────┼────────────────────┤
│ DeepSeek-V3                       │             176 │                 86 │
│ mistral                           │             140 │                 60 │
│ Llama-3.3-70B-Instruct-Turbo-Free │             105 │                 67 │
│ gemini                            │             161 │                 58 │
└───────────────────────────────────┴─────────────────┴────────────────────┘

In [21]:
duckdb.sql(
    "SELECT name as edition, generate_subscripts(problems, 1) - 1 AS problem_index, unnest(problems, recursive := true) FROM statements_nested"
)

┌─────────────┬───────────────┬─────────────────────────────────┬───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────

In [6]:
duckdb.sql("desc solutions")

┌───────────────┬─────────────┬─────────┬─────────┬─────────┬─────────┐
│  column_name  │ column_type │  null   │   key   │ default │  extra  │
│    varchar    │   varchar   │ varchar │ varchar │ varchar │ varchar │
├───────────────┼─────────────┼─────────┼─────────┼─────────┼─────────┤
│ reasoning     │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ python_code   │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ answer        │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ edition       │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ problem_index │ BIGINT      │ YES     │ NULL    │ NULL    │ NULL    │
│ llm           │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
└───────────────┴─────────────┴─────────┴─────────┴─────────┴─────────┘

In [9]:
len(solutions.filter("llm = 'mistral'"))

300

In [10]:
duckdb.sql("desc statements")

┌─────────────┬───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┬─────────┬─────────┬─────────┬─────────┐
│ column_name │                                                                              column_type                                                                              │  null   │   key   │ default │  extra  │
│   varchar   │                                                                                varchar                                                                                │ varchar │ varchar │ varchar │ varchar │
├─────────────┼───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┼─────────┼─────────┼─────────┼─────────┤
│ name        │ VARCHAR                                                                                 

In [12]:
duckdb.sql("SELECT name, unnest(problems, recursive := true) from statements")

┌─────────────┬─────────────────────────────────┬───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────